In [2]:
!pip install requests_html

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 50 kB 2.6 MB/s 
     |████████████████████████████████| 83 kB 1.9 MB/s 
     |████████████████████████████████| 106 kB 51.3 MB/s 
     |████████████████████████████████| 140 kB 62.7 MB/s 
     |████████████████████████████████| 127 kB 58.6 MB/s 
  Created wheel for parse: filename=parse-1.19.0-py3-none-any.whl size=24591 sha256=a9db8763e71107e76dd52621fb372529c71155fb451d3a2b48e17471211394e1
  Stored in directory: /root/.cache/pip/wheels/9c/aa/cc/f2228050ccb40f22144b073f15a2c84f11204f29fc0dce028e
Successfully built parse
  Attempting uninstall: urllib3
    Found existing installation: urllib3 1.24.3
    Uninstalling urllib3-1.24.3:
      Successfully uninstalled urllib3-1.24.3


In [3]:
from requests_html import HTMLSession
import pandas as pd
import time

from IPython.display import clear_output

In [4]:
# Create a session object
session = HTMLSession()

In [34]:
def single_news_scraping(url):
  r = session.get(url)
  title = r.html.find('span.headlinebg')[0].text.split('\n')[0]
  news = r.html.find('div.news-content')
  description = news[0].text.split('এ বিভাগের অন্যান্য সংবাদ')[0].split('\n')[0:-2]
  description = ' '.join([n for n in description])
  description = description.replace('নিজস্ব প্রতিবেদক : ', '')\
                            .replace('নিজস্ব প্রতিবেদক: ', '').strip()
  date = news[0].text.split('এ বিভাগের অন্যান্য সংবাদ')[0].split('\n')[-2]\
                      .replace('শেয়ারনিউজ, ', '')\
                      .replace('শেয়ারনিউজ,', '')\
                      .replace('শেয়ার নিউজ, ', '')

  return title, description, date

In [ ]:
single_news_scraping('https://www.sharenews24.com/article/36697/index.html')

In [ ]:
start = time.time()
cnt, page = 1, 950 # 500 page scrapping done. 

titles = []
links = []
dates = []
descriptions = []
urls = []

for i in range(500, page):
  r = session.get('https://www.sharenews24.com/index.php?page=allnews&catid=3&start='+str(i))
  news_url = [u.attrs['href'] for u in r.html.find('span.headlinebg>a')]
  for u in news_url: 
    title, description, date = single_news_scraping(u)
    titles.append(title)
    urls.append(u)
    descriptions.append(description)
    dates.append(date)
    print(f"Page: {i}, News No: {cnt}")
    print(title)
    print(description)
    print(date)
    print("----------------------------------------------")
    cnt += 1
    clear_output(wait=True)

print(f"Time Required: {time.time()-start}")

Page: 751, News No: 5022
ডিবিএর নতুন ভাইস প্রেসিডেন্ট হলেন সাজিদুল ইসলাম
পুঁজিবাজারের শীর্ষ সংগঠন ডিএসই ব্রোকার্স এসোসিয়েশন অব বাংলাদেশের (ডিবিএ) নতুন ভাইস প্রেসিডেন্ট নির্বাচিত হয়েছেন শ্যামল ইক্যুইটি ম্যানেজমেন্ট লিমিটেডের ব্যবস্থাপনা পরিচালক মো. সাজিদুল ইসলাম। এর আগে তিনি সংগঠনটির পরিচালক পদে দায়িত্ব পালন করছিলেন।  এসোসিয়েশন থেকে পাঠানো এক প্রেস বিজ্ঞপ্তিতে জানানো হয়, গত মঙ্গলবার (২৭ অক্টোবর) ভার্চুয়াল মাধ্যমে অনুষ্ঠিত এসোসিয়েশনের ৭৩তম পর্ষদ সভায় উপস্থিত সদস্যদের সর্বসম্মতিক্রমে তাকে চলতি মেয়াদের জন্যে ভাইস প্রেসিডেন্ট নির্বাচিত করা হয়।  উল্লেখ্য যে, ইতিপূর্বে ঢাকা ব্যাংক সিকিউরিটিজ মোহাম্মদ আলী, এফসিএকে তার দায়িত্ব থেকে প্রত্যাহার করে নিলে এসোসিয়েশনে তার পরিচালক ও ভাইস প্রেসিডেন্ট পটি শূণ্য হয়।
শেয়ারনিউজ; ২৮ অক্টোবর ২০২০
----------------------------------------------


In [11]:
print(news_url[17])

https://www.sharenews24.com/article/36698/index.html


In [ ]:
len(titles), i, cnt

(11, 500, 12)

In [ ]:

# Create a dataframe and export to csv
df = pd.DataFrame({'Title': titles, 'Link': urls, 'Description': descriptions, 'Date': dates, })
df.to_csv('news.csv', index=False)

In [ ]:
from google.colab import data_table
data_table.DataTable(df, include_index=True, num_rows_per_page=10)